# Which Models Support Tool calling?
(same link as last notebook)

https://python.langchain.com/v0.2/docs/concepts/#functiontool-calling

Tool calling is not universal, but is supported by many popular LLM providers, including Anthropic, Cohere, Google, Mistral, OpenAI, and even for locally-running models via Ollama.

LangChain provides a standardized interface for tool calling that is consistent across different models.



The standard interface consists of:

1) ```ChatModel.bind_tools()```: a method for specifying which tools are available for a model to call. This method accepts LangChain tools as well as Pydantic objects.
2) ```AIMessage.tool_calls```: an attribute on the AIMessage returned from the model for accessing the tool calls requested by the model.

# ChatModel.bind_tools()

A method for specifying which tools are available for a model to call. This method accepts LangChain tools as well as Pydantic objects.


https://python.langchain.com/v0.2/docs/concepts/#tools

# Tools
Tools are utilities designed to be called by a model: their inputs are designed to be generated by models, and their outputs are designed to be passed back to models. Tools are needed whenever you want a model to control parts of your code or call out to external APIs.

# What does a tool consists of?

1. The name of the tool.
2. A description of what the tool does.
3. A JSON schema defining the inputs to the tool.
4. A function (and, optionally, an async variant of the function).

# How is a tool bound to a model?

When a tool is bound to a model, the name, description and JSON schema are provided as context to the model. 

Given a list of tools and a set of instructions, a model can request to call one or more tools with specific inputs. Typical usage may look like the following:



```python
tools = [...] # Define a list of tools
llm_with_tools = llm.bind_tools(tools)
ai_msg = llm_with_tools.invoke("do xyz...")
# -> AIMessage(tool_calls=[ToolCall(...), ...], ...)
```

The ```AIMessage``` returned from the model MAY have ```tool_calls``` associated with it. Read this guide for more information on what the response type may look like.

https://python.langchain.com/v0.2/docs/concepts/#aimessage


**AIMessage**


This represents a message with role "assistant". In addition to the content property, these messages also have:

**```response_metadata```**

The ```response_metadata``` property contains additional metadata about the response. 

The data here is often specific to each model provider. This is where information like log-probs and token usage may be stored.

**```tool_calls```**

These represent a decision from an language model to call a tool. They are included as part of an ```AIMessage``` output. They can be accessed from there with the ```.tool_calls``` property.

This property returns a list of ```ToolCalls```. A ```ToolCall``` is a **dictionary** with the following arguments:

* ```name```: The name of the tool that should be called.
* ```args```: The arguments to that tool.
* ```id```: The id of that tool call.



# How to invoke the tools and pass back response to the model?

Once the chosen tools are invoked, the results can be passed back to the model so that it can complete whatever task it's performing

There are generally two different ways to invoke the tool and pass back the response:




# 1) Invoke with just the arguments

### Method-1:

When you invoke a tool with just the arguments (```tool_call["args"]```), you will get back the raw **```tool output```** (usually a string). 

In the example below - the output of invokig the tool is **```tool output```**.

You can then create **```tool message```** from it.


```python
# You will want to previously check that the LLM returned tool calls

tool_call = ai_msg.tool_calls[0]
# ToolCall(args={...}, id=..., ...)

tool_output = tool.invoke(tool_call["args"])

tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call["id"],
    name=tool_call["name"]
)
```

Above content field will generally be passed back to the model. 


### Method-2:

If you do not want the raw tool response to be passed to the model, but you still want to keep it around, you can transform the tool output but also pass it as an artifact (read more about ```ToolMessage.artifact```
https://python.langchain.com/v0.2/docs/concepts/#toolmessage


### ToolMessage
This represents a message with role "tool", which contains the result of calling a tool. In addition to ```role``` and ```content```, this message has:

* a ```tool_call_id``` field which conveys the id of the call to the tool that was called to produce this result.
* an ```artifact``` field which can be used to pass along arbitrary artifacts of the tool execution which are useful to track but which should not be sent to the model.



```python
... # Same code as above
response_for_llm = transform(response)
tool_message = ToolMessage(
    content=response_for_llm,
    tool_call_id=tool_call["id"],
    name=tool_call["name"],
    artifact=tool_output
)
```







# Invoke with ToolCall

The other way to invoke a tool is to call it with the full ```ToolCall``` that was generated by the model. When you do this, the tool will return a ToolMessage. The benefits of this are that you don't have to write the logic yourself to transform the tool output into a ToolMessage. This generally looks like:



```python
tool_call = ai_msg.tool_calls[0]
# -> ToolCall(args={...}, id=..., ...)
tool_message = tool.invoke(tool_call)
# -> ToolMessage(
    content="tool result foobar...",
    tool_call_id=...,
    name="tool_name"
)
```

# Best practices
When designing tools to be used by a model, it is important to keep in mind that:

* Chat models that have explicit tool-calling APIs will be better at tool calling than non-fine-tuned models.
Tool calling API:
https://python.langchain.com/v0.2/docs/concepts/#functiontool-calling

* Models will perform better if the tools have well-chosen names, descriptions, and JSON schemas. This another form of prompt engineering.
* Simple, narrowly scoped tools are easier for models to use than complex tools.


### Tools "How-to" guide:
https://python.langchain.com/v0.2/docs/concepts/#functiontool-calling


### Predefined Tools:
https://python.langchain.com/v0.2/docs/concepts/#functiontool-calling
